In [0]:

# !pip install -q glove-python

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
# drive.mount("/content/drive", force_remount=True)

In [0]:
from glove import Corpus, Glove

import pandas as pd
import numpy as np

In [0]:
def Load_PreTrained_Glove(GloveFile):
  Glove_Dictionary = {}
  with open(GloveFile,'r') as file:
    for line in file:
      tokens = line.split()
      word = tokens[0]
      if word.isalnum():
        word_embedding = np.array([float(val) for val in tokens[1:]])
        Glove_Dictionary[word] = word_embedding
  return Glove_Dictionary

In [0]:
def Calculate_Glove_Embedding(Glove_Dictionary, text):
  
  # print(text)

  words = text.split()
  text_embeddings = []
  
  for word in words:
    ## Ignore if word that is not present in vocabulary appears in the text
    if word in Glove_Dictionary:
      word_embedding = Glove_Dictionary[word]
      text_embeddings.append(word_embedding)
  
  if(len(text_embeddings)>1):
    text_embeddings = np.array(text_embeddings)
    text_avg_embedding = np.mean(text_embeddings, axis=0)
    # print("text_avg_embedding: ", text_avg_embedding)
    # print("IF text_avg_embedding shape: ", text_avg_embedding.shape)
  else:
    text_avg_embedding = np.array(text_embeddings).flatten()
    # text_avg_embedding = text_embeddings[0]
    # print("text_avg_embedding: ", text_avg_embedding)
    # print("ELSE text_avg_embedding shape: ", text_avg_embedding.shape)
  
  return text_avg_embedding

In [0]:
def Get_Glove_Embedding(GloveModel_filepath, Input_Data_filepath, colname="comment_text", Label_Cols=[], Output_Data_filepath=""):
  
  Glove_Dictionary = Load_PreTrained_Glove(GloveModel_filepath)
    
  data = pd.read_csv(Input_Data_filepath)

  data.dropna()
  data = data.dropna(subset=[colname])
  
  Data_Embeddings = []
  for row in range(data.shape[0]):
    text = data.iloc[row][colname]
    text_avg_embedding = Calculate_Glove_Embedding(Glove_Dictionary, text)
    # print("text_avg_embedding shape: ", text_avg_embedding.shape)
    Data_Embeddings.append(text_avg_embedding)
  
  Data_Id = data[["id"]]
  Data_Text = data[[colname]]
  Data_Labels = data[Label_Cols]

  # if(len(Data_Embeddings)==0):
  #   temp_Embeddings = [0 for i in range(300)]
  #   Data_Embeddings.append(temp_Embeddings)

  # Data_Embeddings = np.array(Data_Embeddings)
  Data_Embeddings = pd.DataFrame(Data_Embeddings)
  
  Text_Embedding_Map = pd.concat([Data_Id, Data_Text, Data_Embeddings, Data_Labels], axis=1)
  
  if Output_Data_filepath:
    Text_Embedding_Map.to_csv(Output_Data_filepath)
  
  return Text_Embedding_Map

In [0]:
Label_Cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

Text_Embedding_Map = Get_Glove_Embedding("/content/drive/My Drive/glove.6B.300d.txt", "/content/cleanTrain.csv", "comment_text", 
                                         Label_Cols, "/content/Train_Glove_Embeddings.csv")

In [0]:
Text_Embedding_Map = Get_Glove_Embedding("/content/drive/My Drive/glove.6B.300d.txt", "/content/CleanTest.csv", "comment_text", 
                                         Label_Cols, "/content/Test_Glove_Embeddings.csv")
print(Text_Embedding_Map.head)

<bound method NDFrame.head of                      id  ... identity_hate
0      0001ea8717f6de06  ...             0
1      000247e83dcc1211  ...             0
2      0002f87b16116a7f  ...             0
3      0003e1cccfd5a40a  ...             0
4      00059ace3e3e9a53  ...             0
...                 ...  ...           ...
63561  fff8f64043129fa2  ...             0
63562  fff9d70fe0722906  ...             0
63563  fffa8a11c4378854  ...             0
63564  fffac2a094c8e0e2  ...             0
63565  fffb5451268fb5ba  ...             0

[63566 rows x 308 columns]>
